# **PyTorch ANN Project - Fashion Data**

## **Import Necessary Packages**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt

## **Download Train and Test Data from Open Datasets**

In [ ]:
training_data = datasets.FashionMNIST(root = 'data',
                                      train = True,
                                      transform = ToTensor(),
                                      download = True)

test_data = datasets.FashionMNIST(root = 'data',
                                  train = False,
                                  transform = ToTensor(),
                                  download = True)


In [ ]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

## **Load Data - DataLoader**

In [ ]:
train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

In [ ]:
train_dataloader

In [ ]:
for X,y in test_dataloader:
  # print(X)
  # print(y)
  print("Shape of X : ", X.shape)
  print("Shape of y : ", y.shape)

  break

Shape of X :  torch.Size([64, 1, 28, 28])
Shape of y :  torch.Size([64])


## **Define Model**

In [ ]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),
                                           nn.ReLU(),
                                           nn.Linear(512, 512),
                                           nn.ReLU(),
                                           nn.Linear(512, 10))

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr =1e-3)

## **Train Function**

In [ ]:
def train(dataloader, model, loss_fn, optimizer):

  size = len(dataloader.dataset)

  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y  = X, y

    # COMPUTE PREDICTION ERROR
    pred = model(X)
    loss = loss_fn(pred, y)

    # BACKPROPAGATION
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss = loss.item()
      current = batch*len(X)
      print(f"loss : {loss :>7f} [{current:>5d}/{size :> 5d}]")

## **Test Function**

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for X, y  in dataloader:
      X = X
      y = y
      pred = model(X)
      test_loss = test_loss + (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size

  print(f"Test Error \n Accuracy: {(100*correct):> 0.1f}%, Avg Loss : {test_loss:>8f} \n")

In [ ]:
epochs = 5

for t in range(epochs):
  print(f"Epoch {t+1} \n--------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done")

Epoch 1 
--------------------------
loss : 2.303219 [    0/ 60000]
loss : 2.294371 [ 6400/ 60000]
loss : 2.269730 [12800/ 60000]
loss : 2.261611 [19200/ 60000]
loss : 2.259357 [25600/ 60000]
loss : 2.224532 [32000/ 60000]
loss : 2.232026 [38400/ 60000]
loss : 2.198864 [44800/ 60000]
loss : 2.206340 [51200/ 60000]
loss : 2.170988 [57600/ 60000]
Test Error 
 Accuracy:  0.0%, Avg Loss : 31.038217 

Epoch 2 
--------------------------
loss : 2.175792 [    0/ 60000]
loss : 2.167881 [ 6400/ 60000]
loss : 2.112525 [12800/ 60000]
loss : 2.126110 [19200/ 60000]
loss : 2.078890 [25600/ 60000]
loss : 2.027006 [32000/ 60000]
loss : 2.052560 [38400/ 60000]
loss : 1.976638 [44800/ 60000]
loss : 1.992161 [51200/ 60000]
loss : 1.915531 [57600/ 60000]
Test Error 
 Accuracy:  0.0%, Avg Loss : 37.254777 

Epoch 3 
--------------------------
loss : 1.942960 [    0/ 60000]
loss : 1.911435 [ 6400/ 60000]
loss : 1.798558 [12800/ 60000]
loss : 1.834303 [19200/ 60000]
loss : 1.725176 [25600/ 60000]
loss : 1.68

# **Save Model**

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PYtorch Model")

Saved PYtorch Model


# **Loas Model**

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

# **Prediction**

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
